In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [106]:
retailers = list(range(1, 24))
to_share = ['REGION1', 'REGION2', 'REGION3', 'G_A', 'G_B', 'DELIVERY', 'SPIRIT']
region = dict(zip(retailers, [1 for i in retailers[:8]] + [2 for i in retailers[8:18]] + [3 for i in retailers[18:]]))
region1 = [r for r in retailers[:8]]
region2 = [r for r in retailers[8:18]]
region3 = [r for r in retailers[18:]]

growth = dict(zip(retailers, ['A', 'A','A','B','A','A','B','B','B','A','B','B',
                              'B','B','A','B','B','B','B','A','B','B','B',]))
group_a = [r for r in retailers if growth[r] == 'A']
group_b = [r for r in retailers if growth[r] == 'B']
sum_group_a = len(group_a)
sum_group_b = len(group_b)

deliv = dict(zip(retailers, [11, 47, 44, 25, 10, 26, 26, 54, 18, 51, 20, 105, 
                             7, 16, 34, 100, 50, 21, 11, 19, 14, 10, 11]))
sum_deliv = sum(deliv[i] for i in retailers)

oil_m = dict(zip(retailers, [9, 13, 14, 17, 18, 19, 23, 21, 9, 11, 17, 18, 18,
                             17, 22, 24, 36, 43, 6, 15, 15, 25, 39]))
sum_region1 = sum(oil_m[i] for i in region1)
sum_region2 = sum(oil_m[i] for i in region2)
sum_region3 = sum(oil_m[i] for i in region3)

spi_m = dict(zip(retailers, [34, 411, 82, 157, 5, 183, 14, 215, 102, 21, 54, 0,
                             6, 96, 118, 112, 535, 8, 53, 28, 69, 65, 27]))
sum_spi_m = sum(spi_m[i] for i in retailers)

five_perc = dict(zip(to_share, [sum_region1*0.05,
                                sum_region2*0.05,
                                sum_region3*0.05,
                                sum_group_a*0.05,
                                sum_group_b*0.05,
                                sum_deliv*0.05,
                                sum_spi_m*0.05]))


In [107]:
model = gp.Model('Market Sharing')

# add vars
x = model.addVars(retailers,          # = 1 if retailer belongs to D1
                  vtype=GRB.BINARY,
                  name='x')
sl = model.addVars(to_share,
                   name='slack')
su = model.addVars(to_share,
                   name='surplus')

for s in to_share:
    sl[s].ub = five_perc[s]
    su[s].ub = five_perc[s]

# objective function
model.setObjective(gp.quicksum(sl[s] for s in to_share) 
                   + gp.quicksum(su[s] for s in to_share))

# add constraints
# share delivery points
model.addConstr((gp.quicksum(deliv[r]*x[r] for r in retailers) 
                  + sl['DELIVERY'] - su['DELIVERY']) == sum_deliv*0.4)

# share spirit market
model.addConstr((gp.quicksum(spi_m[r]*x[r] for r in retailers) 
                  + sl['SPIRIT'] - su['SPIRIT']) == sum_spi_m*0.4)

# share oil market region 1
model.addConstr((gp.quicksum(oil_m[r]*x[r] for r in region1) 
                  + sl['REGION1'] - su['REGION1']) == sum_region1*0.4)

# share oil market region 2
model.addConstr((gp.quicksum(oil_m[r]*x[r] for r in region2) 
                  + sl['REGION2'] - su['REGION2']) == sum_region2*0.4)

# share oil market region 3
model.addConstr((gp.quicksum(oil_m[r]*x[r] for r in region3) 
                  + sl['REGION3'] - su['REGION3']) == sum_region3*0.4)

# number retailers in group a
model.addConstr((gp.quicksum(x[r] for r in group_a)
                  + sl['G_A'] - su['G_A']) == sum_group_a*0.4)

# number retailers in group b
model.addConstr((gp.quicksum(x[r] for r in group_b) 
                  + sl['G_B'] - su['G_B']) == sum_group_b*0.4)

model.update()
model.write('Market Sharing.lp')

In [108]:
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 37 columns and 105 nonzeros
Model fingerprint: 0xf862bef8
Variable types: 14 continuous, 23 integer (23 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e-01, 1e+02]
  RHS range        [3e+00, 1e+03]
Presolve time: 0.00s
Presolved: 7 rows, 37 columns, 105 nonzeros
Variable types: 14 continuous, 23 integer (23 binary)

Root relaxation: objective 0.000000e+00, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    7          -    0.00000      -     -    0s
H    0     0                     131.6000000    0.00000   100%     -    0s
H    0     0                     111.6000000    0.00000   100%     -    0s
H

In [109]:
for s in to_share:
    print(sl[s].x)

0.0
1.999999999999968
0.9999999999999805
0.2000000000000003
0.0
1.9999999999999112
0.999999999999627


In [110]:
print('Retailers in D1')
for r in retailers:
    if x[r].x>0:
        print(r)

print('\nRetailers in D2')
for r in retailers:
    if x[r].x<1:
        print(r)

Retailers in D1
2
6
7
9
12
13
14
15
23

Retailers in D2
1
3
4
5
8
10
11
16
17
18
19
20
21
22
